In [1]:
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [2]:
url = 'https://www.hasznaltauto.hu/talalatilista/PDNG2VG3R2NTAEH5C57QDBFEXN2R4K2V5JFX4AKNZJABYXZGWIG5UEHF345QNOUL3E6DCOTHFZTS4BTEKXEUPEHFJYFA2EJ36IFE4OEIENR2KFDBISF6MDT5UQE5USYBEZ3TYMP6F3CZSBQVGHDBSSFRANDDJYDLVNOIULNECBNQMPWWZRAQV3D5TXCZ2XQRFHWLJSFZIAMB3GCYL7A5W7WKENCTS7O7UTUMBA73VPAKR4GCMGLRUBCD5YC4ONOP5DIVYMGSBM5FKVV6MFCWDAEGGWEOGWIWDFB37YI24EFIWVXOWJS7T3H3U6Q26KJN3CVNWNPYX5JW5RLD5BGQZKL2YUVIUYWS6G7NL4JL7D3HGKIWXSDFUNNLFH7WJFA2GRTIEF57VBXH3OFC3HWASB3ODTRIWDL6GLMQMWNFK3UQZMSEHTSZN3ZHIZW3PGJSFJF4KMT3XY6XA57I6IFGADG4RDXQZZCBH3KDZ5DJFVFJNM55JP3G5NVLMRL3HPKP6Z7LMD7C7BSIGR3ZIHL2D375W47HGLV7KXHMBWZFLXZQCWOY6TKBWDHYTXRJ46GGA3WUQOWJM7L4ONH66S6CYRCYF2WFRHBJQRI7GDPKGX7EGCQVVBKR625QMQKA32XIZFJKRYLMEAFUUNKMZ5L6LVA24YN5ORSZ3C4ENNFXUQT2WMF5CX2IGGRVXH53BZTYJYMG4G7RAM43VEFHF3F4F67DDTUO4GHKSZ6APRTGRLPONHYDH7NC75HZ7QDUZ4TQCI'

In [3]:
out = []
maxpage = int(BeautifulSoup(requests.get(url).content,'html5lib').find('ul',{'class':'pagination'})\
            .find('li',{'class':'last'}).text.strip())

In [ ]:
for i in tqdm(range(1,maxpage+1)):
    soup = BeautifulSoup(requests.get(url + '/page%d' % i).content,'html5lib')
    for car in soup.find_all('div',{'class':'talalati-sor'}):
        dic = {'nev':car.find('h3').text.strip(),
               'link':car.find('h3').find('a')['href'],
              'price':car.find('div',{'class':'vetelar'}).text.strip()}
        for aind,s in enumerate(car.find('div',{'class':'adatok'}).find_all('span')):
            dic['adat-%d' % aind] = s.text.strip()
        cimkelista = car.find('div',{'cimke-lista'})
        if type(cimkelista) != type(None):
            for cimke in cimkelista.find_all('span'):
                dic[cimke.text] = 1
        out.append(dic.copy())

In [ ]:
pd.DataFrame(out).drop_duplicates().to_csv('data/hasznaltauto-%s.csv' % str(datetime.date.today()),index=None)+"_hahu"

In [1]:
url = 'https://www.hasznaltauto.hu/szemelyauto/citroen/c3_aircross/citroen_c3_aircross_feel_1_2pt_82le-12799518'

In [9]:
def get_details(url):
    soup = BeautifulSoup(requests.get(url).content,'html5lib')

    out = {'leiras':soup.find('div',{'class':'leiras'}).text.strip(),
          'url':url}
    for tr in soup.find('table',{'class':'hirdetesadatok'}).find_all('tr'):
        tds = tr.find_all('td')
        if len(tds) > 1:
            out[tds[0].text.strip()] = tds[1].text.strip()

    return out

In [10]:
get_details(url)

{'leiras': 'Leírás\n                Egyedi ajánlatainkért hívjon, foglaljon időpontot és személyesen megbeszéljük. A hirdetés nem minősül hivatalos ajánlattételnek. A jelenlegi kép csak illusztráció. Az akció részleteiről és feltételeiről keressen minket bizalommal. Szalonunk hétfőtől péntekig 8-17 óra között, szombaton 8-12 óra között tart nyitva.',
 'Vételár:': '4\xa0710\xa0000\xa0Ft',
 'Vételár EUR:': '€\xa014\xa0922',
 'Átvehető:': '2019.',
 'Évjárat:': '2018/4',
 'Állapot:': 'Kitűnő',
 'Kivitel:': 'Városi terepjáró (crossover)',
 'Kilométeróra állása:': '5 km',
 'Ajtók száma:': '5',
 'Szín:': 'Fehér',
 'Kárpit színe (1):': 'Fekete',
 'Klíma fajtája:': 'Manuális klíma',
 'Üzemanyag:': 'Benzin',
 'Hengerűrtartalom:': '1\xa0199 cm³',
 'Teljesítmény:': '60 kW, 82 LE',
 'Sebességváltó fajtája:': 'Manuális (5 fokozatú) sebességváltó',
 'Okmányok jellege:': 'Okmányok nélkül',
 'Műszaki vizsga érvényes:': '2018/5',
 'Nyári gumi méret:': '195/55\xa0R\xa016'}